# Collection Success Rate Analysis

In [84]:
import pandas as pd
import boto3
from tqdm import tqdm

In [148]:
# for domestic sites
total_domestic_sites = 45

def update_collection_rate_stats(local_stats_path, s3_bucket, prefix, total_sites):
    df = pd.read_csv(local_stats_path, dtype={'time': str})

    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=s3_bucket, Prefix=prefix, Delimiter="/")
    
    # Iterate through the subfolders
    new_data = []
    collected_time = df["time"].to_list()
    for folder_name in tqdm(response.get('CommonPrefixes', [])):
        folder_name = folder_name["Prefix"]
        time = folder_name.split("/")[-2]

        # check if datapoint is collected
        if time in collected_time:
            continue

        # not collected
        datapoint_res = s3.list_objects_v2(Bucket=s3_bucket, Prefix=folder_name + "html/")
        collectionn_rate = len(datapoint_res["Contents"]) / total_domestic_sites
        new_data.append({"time" : str(time), "collection_rate" : collectionn_rate})
    
    new_df = pd.concat([df, pd.DataFrame(new_data)], ignore_index=True)
    new_df.to_csv(local_stats_path, index=False) # update saved data

    return new_df



In [149]:
t = update_collection_rate_stats("./data/collection_rate_domestic.csv", "news-collection-2024-3371", "current_data/", total_domestic_sites)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 439/439 [00:13<00:00, 33.72it/s]
